<a href="https://www.kaggle.com/code/yutodennou/competition-lightgbm-and-nn?scriptVersionId=155137055" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🗃️Import Libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import tensorflow_probability as tfp

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


# 📏Preprocessing

In [2]:
train_dataset = pd.read_csv('/kaggle/input/playground-series-s3e25/train.csv')

In [3]:
features = ['allelectrons_Total', 'density_Total', 'allelectrons_Average',
            'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
            'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
            'zaratio_Average', 'density_Average', 'Hardness']

In [4]:
X = train_dataset[features].drop(columns='Hardness')
y = train_dataset.Hardness

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
model_pre = LGBMRegressor(max_depth=5,n_estimators=250)
model_pre.fit(X, y)
X_new = X.copy()
X_new['Hardness_pred'] = model_pre.predict(X)

In [7]:
def loss_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=50)

def metric_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=100) - tfp.stats.percentile(tf.abs(y_true - y_pred), q=0)

In [8]:
callbacks_list1 = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00001),
    tf.keras.callbacks.TerminateOnNaN()
] 
callbacks_list2 = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00002),
    tf.keras.callbacks.TerminateOnNaN()
] 
callbacks_list3 = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00003),
    tf.keras.callbacks.TerminateOnNaN()
] 

In [9]:
def create_model(hp_learning_rate):
    
    input_layer = tf.keras.Input(shape=(len(features), ))
    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    output_layer = tf.keras.layers.Dense(1)(x)    
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(hp_learning_rate, beta_1=0.5),
                  loss=loss_fn,
                  metrics=metric_fn)
    
    return model

# 🔍Learning

In [10]:
model1 = create_model(0.013)
history1 = model1.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list1,
                    validation_split=0.1)

Epoch 1/100
293/293 [==============================] - 7s 6ms/step - loss: 1.2518 - metric_fn: 4.6548 - val_loss: 1.3749 - val_metric_fn: 3.8326 - lr: 0.0130
Epoch 2/100
293/293 [==============================] - 1s 5ms/step - loss: 0.8594 - metric_fn: 3.2234 - val_loss: 0.5304 - val_metric_fn: 3.1186 - lr: 0.0130
Epoch 3/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6768 - metric_fn: 3.0339 - val_loss: 0.5001 - val_metric_fn: 3.2081 - lr: 0.0130
Epoch 4/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6191 - metric_fn: 3.0970 - val_loss: 0.9856 - val_metric_fn: 3.4738 - lr: 0.0130
Epoch 5/100
293/293 [==============================] - 2s 5ms/step - loss: 0.5984 - metric_fn: 3.1353 - val_loss: 0.4354 - val_metric_fn: 3.2460 - lr: 0.0130
Epoch 6/100
293/293 [==============================] - 1s 5ms/step - loss: 0.5516 - metric_fn: 3.2121 - val_loss: 0.5740 - val_metric_fn: 3.3390 - lr: 0.0130
Epoch 7/100
293/293 [==============================]

In [11]:
model2 = create_model(0.009)
history2 = model2.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=150,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list2,
                    validation_split=0.1)

Epoch 1/150
293/293 [==============================] - 4s 6ms/step - loss: 1.3261 - metric_fn: 4.9934 - val_loss: 0.6461 - val_metric_fn: 3.1225 - lr: 0.0090
Epoch 2/150
293/293 [==============================] - 1s 5ms/step - loss: 0.8603 - metric_fn: 3.3325 - val_loss: 0.6390 - val_metric_fn: 3.2261 - lr: 0.0090
Epoch 3/150
293/293 [==============================] - 1s 5ms/step - loss: 0.6891 - metric_fn: 3.0388 - val_loss: 0.7059 - val_metric_fn: 3.2196 - lr: 0.0090
Epoch 4/150
293/293 [==============================] - 1s 5ms/step - loss: 0.6642 - metric_fn: 3.1301 - val_loss: 0.5644 - val_metric_fn: 3.6221 - lr: 0.0090
Epoch 5/150
293/293 [==============================] - 1s 5ms/step - loss: 0.6329 - metric_fn: 3.2585 - val_loss: 0.6794 - val_metric_fn: 3.4601 - lr: 0.0090
Epoch 6/150
293/293 [==============================] - 1s 5ms/step - loss: 0.5865 - metric_fn: 3.1656 - val_loss: 0.5426 - val_metric_fn: 3.3108 - lr: 0.0090
Epoch 7/150
293/293 [==============================]

In [12]:
model3 = create_model(0.010)
history3 = model3.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=150,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list3,
                    validation_split=0.1)

Epoch 1/150
293/293 [==============================] - 4s 6ms/step - loss: 1.2103 - metric_fn: 4.7267 - val_loss: 1.4132 - val_metric_fn: 4.0856 - lr: 0.0100
Epoch 2/150
293/293 [==============================] - 1s 5ms/step - loss: 0.7992 - metric_fn: 3.1173 - val_loss: 0.5904 - val_metric_fn: 3.3098 - lr: 0.0100
Epoch 3/150
293/293 [==============================] - 1s 5ms/step - loss: 0.7381 - metric_fn: 3.2691 - val_loss: 1.0077 - val_metric_fn: 4.1363 - lr: 0.0100
Epoch 4/150
293/293 [==============================] - 1s 5ms/step - loss: 0.7024 - metric_fn: 3.3521 - val_loss: 0.5685 - val_metric_fn: 3.6566 - lr: 0.0100
Epoch 5/150
293/293 [==============================] - 2s 5ms/step - loss: 0.6331 - metric_fn: 3.2314 - val_loss: 0.5322 - val_metric_fn: 3.4489 - lr: 0.0100
Epoch 6/150
293/293 [==============================] - 1s 5ms/step - loss: 0.5872 - metric_fn: 3.0089 - val_loss: 0.5952 - val_metric_fn: 3.3777 - lr: 0.0100
Epoch 7/150
293/293 [==============================]

In [13]:
model4 = create_model(0.010)
history4 = model4.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list1,
                    validation_split=0.1)

Epoch 1/100
293/293 [==============================] - 4s 6ms/step - loss: 1.2791 - metric_fn: 4.7010 - val_loss: 1.2815 - val_metric_fn: 3.9174 - lr: 0.0100
Epoch 2/100
293/293 [==============================] - 1s 5ms/step - loss: 0.9367 - metric_fn: 3.2214 - val_loss: 0.5762 - val_metric_fn: 3.2077 - lr: 0.0100
Epoch 3/100
293/293 [==============================] - 1s 5ms/step - loss: 0.8200 - metric_fn: 3.1798 - val_loss: 0.7148 - val_metric_fn: 3.4388 - lr: 0.0100
Epoch 4/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6991 - metric_fn: 3.0290 - val_loss: 0.5102 - val_metric_fn: 3.1789 - lr: 0.0100
Epoch 5/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6510 - metric_fn: 3.1136 - val_loss: 0.6666 - val_metric_fn: 3.4288 - lr: 0.0100
Epoch 6/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6322 - metric_fn: 3.1221 - val_loss: 0.6066 - val_metric_fn: 3.3630 - lr: 0.0100
Epoch 7/100
293/293 [==============================]

In [14]:
model5 = create_model(0.012)
history5 = model5.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list2,
                    validation_split=0.1)

Epoch 1/100
293/293 [==============================] - 4s 6ms/step - loss: 1.3118 - metric_fn: 6.7665 - val_loss: 0.7390 - val_metric_fn: 3.5599 - lr: 0.0120
Epoch 2/100
293/293 [==============================] - 1s 5ms/step - loss: 0.8587 - metric_fn: 3.4348 - val_loss: 0.5306 - val_metric_fn: 3.2579 - lr: 0.0120
Epoch 3/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6959 - metric_fn: 3.1210 - val_loss: 0.5225 - val_metric_fn: 3.3654 - lr: 0.0120
Epoch 4/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6919 - metric_fn: 3.2721 - val_loss: 0.6399 - val_metric_fn: 3.2228 - lr: 0.0120
Epoch 5/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6041 - metric_fn: 3.1685 - val_loss: 0.4655 - val_metric_fn: 3.3432 - lr: 0.0120
Epoch 6/100
293/293 [==============================] - 1s 5ms/step - loss: 0.5712 - metric_fn: 3.1765 - val_loss: 0.5168 - val_metric_fn: 3.3837 - lr: 0.0120
Epoch 7/100
293/293 [==============================]

In [15]:
model6 = create_model(0.013)
history6 = model6.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list3,
                    validation_split=0.1)

Epoch 1/100
293/293 [==============================] - 4s 6ms/step - loss: 1.3541 - metric_fn: 4.2934 - val_loss: 0.8715 - val_metric_fn: 3.3651 - lr: 0.0130
Epoch 2/100
293/293 [==============================] - 1s 5ms/step - loss: 0.8528 - metric_fn: 3.0445 - val_loss: 0.6197 - val_metric_fn: 3.2152 - lr: 0.0130
Epoch 3/100
293/293 [==============================] - 1s 5ms/step - loss: 0.7683 - metric_fn: 3.0652 - val_loss: 0.8970 - val_metric_fn: 3.4410 - lr: 0.0130
Epoch 4/100
293/293 [==============================] - 1s 5ms/step - loss: 0.7192 - metric_fn: 3.0664 - val_loss: 0.6553 - val_metric_fn: 3.2257 - lr: 0.0130
Epoch 5/100
293/293 [==============================] - 2s 5ms/step - loss: 0.6786 - metric_fn: 3.1300 - val_loss: 0.5572 - val_metric_fn: 3.6056 - lr: 0.0130
Epoch 6/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6267 - metric_fn: 3.2325 - val_loss: 0.5768 - val_metric_fn: 3.3844 - lr: 0.0130
Epoch 7/100
293/293 [==============================]

In [16]:
sample_submission = pd.read_csv('/kaggle/input/playground-series-s3e25/sample_submission.csv')
test_dataset = pd.read_csv('/kaggle/input/playground-series-s3e25/test.csv')

In [17]:

test_dataset['Hardness_pred'] = model_pre.predict(test_dataset.astype('float32').drop(columns='id'))
test_dataset["Hardness"] = model1.predict(test_dataset.astype('float32').drop(columns='id'))*0.3+model2.predict(test_dataset.astype('float32').drop(columns='id'))*0.2+model3.predict(test_dataset.astype('float32').drop(columns='id'))*0.1+model4.predict(test_dataset.astype('float32').drop(columns='id'))*0.2+model5.predict(test_dataset.astype('float32').drop(columns='id'))*0.1+model6.predict(test_dataset.astype('float32').drop(columns='id'))*0.1
sample_submission = test_dataset[['id', "Hardness"]]
sample_submission.to_csv("submission.csv", index=False)

217/217 [==============================] - 0s 1ms/step


In [18]:
sample_submission

,id,Hardness
0,10407,2.513499
1,10408,2.377078
2,10409,5.754146
3,10410,3.828435
4,10411,5.754146
...,...,...
6934,17341,5.754146
6935,17342,3.834049
6936,17343,5.763113
6937,17344,5.425693
